In [6]:
import pandas as pd
import numpy as np
import math
import math
from pathlib import Path

from tournament import Tournament, Participant, Match

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
GAME_ID = [273, 314, 331, 348, 359, 371, 380, 390, 399, 406]
# GAME_ID = [314]

for id in GAME_ID:

    league_settings_query = f"SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week from raw.league_settings where game_id = '{id}'"
    league_settings = (
        DatabaseCursor(PATH, option_schema="raw")
        .copy_data_from_postgres(league_settings_query)
        .drop_duplicates()
    )

    meta_query = "SELECT game_id, league_id, end_week FROM raw.league_metadata"
    meta = DatabaseCursor(PATH).copy_data_from_postgres(meta_query).drop_duplicates()

    league_settings = league_settings.merge(
        meta,
        how="inner",
        left_on=["game_id", "league_id"],
        right_on=["game_id", "league_id"],
    )

    num_playoff_teams = league_settings["num_playoff_teams"].values[0]
    num_conso_teams = league_settings["num_playoff_consolation_teams"].values[0]
    max_teams = league_settings["max_teams"].values[0]
    num_toliet_teams = (
        (
            max_teams - num_playoff_teams
            if num_playoff_teams >= (max_teams / 2)
            else num_playoff_teams
        )
        if num_conso_teams != 0
        else False
    )
    playoff_start_week = league_settings["playoff_start_week"].values[0]
    playoff_end_week = league_settings["end_week"].values[0]
    playoff_weeks = range(playoff_start_week, playoff_end_week + 1)

    playoff_board_query = f"SELECT * FROM dev.playoff_board_{id}"
    playoff_board = (
        DatabaseCursor(PATH, option_schema="dev")
        .copy_data_from_postgres(playoff_board_query)
        .drop_duplicates()
    )

    teams = list(playoff_board["team_key"].head(max_teams).unique())

    playoff_teams = list(
        playoff_board["team_key"][
            playoff_board["reg_season_rank"] <= num_playoff_teams
        ].unique()
    )
    if num_toliet_teams:
        if id == 314:
            conso_teams = list(
                playoff_board["team_key"][
                    playoff_board["reg_season_rank"] <= (max_teams - num_playoff_teams)
                ].unique()
            )
            conso_teams = [team for team in conso_teams if team not in playoff_teams]
            toilet_teams = False

        elif id >= 406:
            conso_teams = list(
                playoff_board["team_key"][
                    playoff_board["reg_season_rank"] > num_playoff_teams
                ].unique()
            )
            toilet_teams = list(
                playoff_board["team_key"][
                    playoff_board["reg_season_rank"] > (max_teams - num_conso_teams)
                ].unique()
            )
            conso_teams = [team for team in conso_teams if team not in toilet_teams]

        else:
            conso_teams = False
            toilet_teams = list(
                playoff_board["team_key"][
                    playoff_board["reg_season_rank"] > (max_teams - num_toliet_teams)
                ].unique()
            )

    else:
        toilet_teams = False
        conso_teams = False

    playoff_end_week_mask = playoff_board["week"] == playoff_end_week

    playoff_bracket = Tournament(playoff_teams)
    num_of_rounds = int(math.ceil(math.log2(len(playoff_teams))))
    for round_ in range(num_of_rounds):
        for match in playoff_bracket.get_active_matches():
            right_comp = match.get_participants()[0].get_competitor()
            right_score = playoff_board["team_pts"][
                (playoff_board["team_key"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            right_pro_score = playoff_board["team_pro_pts"][
                (playoff_board["team_key"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            left_comp = match.get_participants()[1].get_competitor()
            left_score = playoff_board["team_pts"][
                (playoff_board["team_key"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            left_pro_score = playoff_board["team_pro_pts"][
                (playoff_board["team_key"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_)
            ].values[0]
            playoff_board.loc[
                (playoff_board["team_key"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp",
            ] = left_comp
            playoff_board.loc[
                (playoff_board["team_key"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp_pts",
            ] = left_score
            playoff_board.loc[
                (playoff_board["team_key"] == right_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp_pro_pts",
            ] = left_pro_score
            playoff_board.loc[
                (playoff_board["team_key"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp",
            ] = right_comp
            playoff_board.loc[
                (playoff_board["team_key"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp_pts",
            ] = right_score
            playoff_board.loc[
                (playoff_board["team_key"] == left_comp)
                & (playoff_board["week"] == playoff_start_week + round_),
                "po_opp_pro_pts",
            ] = right_pro_score
            print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
            if right_score > left_score:
                match.set_winner(right_comp)
            elif right_score < left_score:
                match.set_winner(left_comp)
    for rk, tm in playoff_bracket.get_final().items():
        playoff_board.loc[
            (playoff_board["team_key"] == tm) & playoff_end_week_mask, "finish"
        ] = int(rk)

    if conso_teams:
        if len(conso_teams) < len(playoff_teams):
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    right_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp",
                    ] = left_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pts",
                    ] = left_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp",
                    ] = right_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pts",
                    ] = right_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    right_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp",
                    ] = left_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pts",
                    ] = left_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp",
                    ] = right_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pts",
                    ] = right_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        for rk, tm in conso_bracket.get_final().items():
            playoff_board.loc[
                (playoff_board["team_key"] == tm) & playoff_end_week_mask, "finish"
            ] = int(rk) + len(playoff_teams)

    if toilet_teams:
        if len(toilet_teams) < len(playoff_teams):
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    right_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    left_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1)
                    ].values[0]
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp",
                    ] = left_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pts",
                    ] = left_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp",
                    ] = right_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pts",
                    ] = right_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_ + 1),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    right_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = playoff_board["team_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    left_pro_score = playoff_board["team_pro_pts"][
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_)
                    ].values[0]
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp",
                    ] = left_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pts",
                    ] = left_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == right_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp",
                    ] = right_comp
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pts",
                    ] = right_score
                    playoff_board.loc[
                        (playoff_board["team_key"] == left_comp)
                        & (playoff_board["week"] == playoff_start_week + round_),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    print(f"{right_comp}: {right_score} vs {left_comp}: {left_score}")
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        for rk, tm in toilet_bracket.get_final().items():
            playoff_board.loc[
                (playoff_board["team_key"] == tm) & playoff_end_week_mask, "finish"
            ] = (
                int(rk) + len(playoff_teams) + (len(conso_teams) if conso_teams else 0)
            )

    playoff_board.loc[playoff_end_week_mask, "finish"] = playoff_board.loc[
        playoff_end_week_mask, "finish"
    ].fillna(playoff_board["reg_season_rank"])

Successfully pulled: SELECT game_id, league_id, max_teams, num_playoff_consolation_teams, num_playoff_teams, playoff_start_week from dev.league_settings where game_id = '273'
Successfully pulled: SELECT game_id, league_id, end_week FROM dev.league_metadata
Successfully pulled: SELECT * FROM prod.playoff_board_273
273.l.777818.t.7: 88.5 vs 273.l.777818.t.1: 65.86
273.l.777818.t.10: 106.34 vs 273.l.777818.t.12: 131.12
273.l.777818.t.2: 71.9 vs 273.l.777818.t.4: 75.38
273.l.777818.t.6: 77.14 vs 273.l.777818.t.9: 63.68
273.l.777818.t.7: 111.66 vs 273.l.777818.t.6: 123.5
273.l.777818.t.12: 110.48 vs 273.l.777818.t.4: 120.76
273.l.777818.t.1: 135.44 vs 273.l.777818.t.9: 90.34
273.l.777818.t.10: 100.7 vs 273.l.777818.t.2: 69.02
273.l.777818.t.6: 96.02 vs 273.l.777818.t.4: 98.96
273.l.777818.t.7: 105.22 vs 273.l.777818.t.12: 112.76
273.l.777818.t.1: 135.6 vs 273.l.777818.t.10: 79.68
273.l.777818.t.9: 110.12 vs 273.l.777818.t.2: 49.1
Successfully pulled: SELECT game_id, league_id, max_teams, nu

In [7]:
playoff_board.sort_values("finish")

,game_id,team_key,team_name,team_nickname,week,team_pts,team_pro_pts,ttl_pts_for_run,ttl_pro_pts_for_run,reg_season_rank,po_opp,po_opp_pts,po_opp_pro_pts,finish
12,406,406.l.103661.t.7,Harris & Harris RBs at Law 👨🏾‍⚖️,Greg,17,162.82,131.13,287.82,254.84,3,406.l.103661.t.1,130.70,125.13,1.0
10,406,406.l.103661.t.1,Fantasy Sidelines,Tim,17,130.70,125.13,245.68,229.00,1,406.l.103661.t.7,162.82,131.13,2.0
13,406,406.l.103661.t.2,Dak to the Future,Wes,17,103.94,124.62,192.44,253.31,4,406.l.103661.t.4,101.16,129.30,3.0
11,406,406.l.103661.t.4,King of the Hill,Ryan,17,101.16,129.30,214.08,261.70,2,406.l.103661.t.2,103.94,124.62,4.0
14,406,406.l.103661.t.6,Wide Out University,Chris,17,119.30,106.31,203.16,186.59,5,406.l.103661.t.3,97.46,121.63,5.0
15,406,406.l.103661.t.3,Fightin’ Blue Cock/Chubb/Wood,Kevin,17,97.46,121.63,254.34,232.60,6,406.l.103661.t.6,119.30,106.31,6.0
17,406,406.l.103661.t.9,Push Ups in Prison,Pat,17,103.08,119.58,227.62,237.26,8,406.l.103661.t.8,102.32,116.64,7.0
16,406,406.l.103661.t.8,Poor Decisions 😢,Pete,17,102.32,116.64,196.70,225.88,7,406.l.103661.t.9,103.08,119.58,8.0
19,406,406.l.103661.t.5,The Titanic,Jeremy,17,140.46,115.56,212.80,219.44,10,406.l.103661.t.10,79.68,112.58,9.0
18,406,406.l.103661.t.10,Team 10,James,17,79.68,112.58,180.38,216.18,9,406.l.103661.t.5,140.46,115.56,10.0
